In [32]:
def read_test():
    path = r"D:\Repos\Minecraft\Forge\SuperFactoryManager\src\gametest\java\ca\teamdman\sfm\ai\TestChambers.java"
    with open(path, "r") as f:
        return f.read()
read_test()

'package ca.teamdman.sfm.ai;\n\nimport ca.teamdman.sfm.SFM;\nimport ca.teamdman.sfm.SFMGameTestBase;\nimport net.minecraft.core.BlockPos;\nimport net.minecraft.gametest.framework.GameTest;\nimport net.minecraft.gametest.framework.GameTestHelper;\nimport net.minecraft.world.entity.item.ItemEntity;\nimport net.minecraft.world.item.ItemStack;\nimport net.minecraft.world.item.Items;\nimport net.minecraft.world.level.block.Blocks;\nimport net.minecraft.world.level.block.DoorBlock;\nimport net.minecraft.world.phys.Vec3;\nimport net.neoforged.gametest.GameTestHolder;\nimport net.neoforged.gametest.PrefixGameTestTemplate;\n\n@GameTestHolder(SFM.MOD_ID)\n@PrefixGameTestTemplate(false)\npublic class TestChambers extends SFMGameTestBase {\n    /**\n     * The agent must make the test case pass.\n     * The agent is provided code at the beginning and the end of the test case.\n     */\n    @GameTest(template = "3x4x3")\n    public static void open_door(GameTestHelper helper) {\n        // begin te

In [33]:
def write_test(content: str):
    path = r"D:\Repos\Minecraft\Forge\SuperFactoryManager\src\gametest\java\ca\teamdman\sfm\ai\TestChambers.java"
    with open(path, "w") as f:
        f.write(content)

In [34]:
def indent_text(text, spaces=4):
    space_str = " " * spaces
    return "\n".join(f"{space_str}{line}" for line in text.split("\n"))
print(indent_text("a\nb\nc",8))

        a
        b
        c


In [55]:
def with_agent_content(test: str, new_content: str) -> str:
    """
    We want to remove any previous attempts at the test

    // begin agent code
    item.setPos(Vec3.atCenterOf(helper.absolutePos(pressurePlatePos).offset(0,3,0)));
    // end agent code

    should remove the content between the two comments
    """
    region_start = "        // begin agent code"
    region_end = "        // end agent code"
    start = test.find(region_start)
    end = test.find(region_end)
    return test[:start+len(region_start)] + "\n" + new_content + ("\n" if new_content != "" else "") + test[end:]

In [36]:
def get_agent_content(test: str) -> str:
    region_start = "        // begin agent code"
    region_end = "        // end agent code"
    start = test.find(region_start)
    end = test.find(region_end)
    return test[start+len(region_start)+1:end]

In [37]:
def focus_intellij():
    import pygetwindow as gw
    try:
        title = next(x for x in gw.getAllTitles() if "TestChambers.java" in x)
        windows = gw.getWindowsWithTitle(title)
        assert len(windows) > 0, f"Window not found: {title}"
        windows[0].activate()
    except Exception as e:
        print(f"Error focusing window: {e}")

In [38]:
def is_building():
    import pyautogui
    return pyautogui.pixel(1947, 622) == (95, 173, 101)

In [39]:
def get_build_output():
    import pyautogui
    import pyperclip
    # store cursor position
    x, y = pyautogui.position()
    # click and drag from 2890, 638 to 2895, 638 to select text without activating links
    pyautogui.moveTo(2890, 638)
    pyautogui.mouseDown()
    pyautogui.moveTo(2895, 638)
    pyautogui.mouseUp()
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.hotkey('ctrl', 'c')
    # restore cursor position
    pyautogui.moveTo(x, y)
    return pyperclip.paste().replace('\r', '')

In [40]:
def is_happy_build_output(output):
    happy = """
Executing pre-compile tasks…
Running 'before' tasks
Checking sources
Running 'after' tasks
Finished, saving caches…
Executing post-compile tasks…
Finished, saving caches…
    """.strip()
    return output.strip() == happy

In [49]:
def hot_reload():
    import pyautogui
    from time import sleep
    print("[Hot Reload] Focusing IntelliJ")
    focus_intellij()
    print("[Hot Reload] Trigger hot reload")
    pyautogui.hotkey('ctrl', 'alt', 'num0')
    print("[Hot Reload] Wait for build to finish")
    while is_building():
        sleep(0.1)
    sleep(1.5)
    print("[Hot Reload] Copy build output")
    output = get_build_output()
    success = is_happy_build_output(output)
    print(f"[Hot Reload] Build output succeeded: {success}")
    return success, output

In [ ]:
from pathlib import Path
msgdir = Path("./messages")
msgdir.mkdir(parents=True, exist_ok=True)
def run_test():
    print("[Run Test] Running test, ensure you ran `/sfm_ai listen` in Minecraft")
    # touch messages/run.txt
    (msgdir / "run.txt").touch()

    from time import sleep
    # wait for the file to have test results appended
    while not (msgdir / "run.txt").stat().st_size > 0:
        sleep(0.1)
    
    # read the file
    with open(msgdir / "run.txt", "r") as f:
        results = f.read()

    archive_content = get_agent_content(read_test()) + "\n\n" + results
    
    # clean up the file by renaming it with the time
    from datetime import datetime
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    (msgdir / f"run_{now}.txt").write_text(archive_content)
    (msgdir / "run.txt").unlink()

    return results

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-13b-Instruct-hf", device_map="auto")
print(tokenizer)

c:\Users\TeamD\.conda\envs\sfm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CodeLlamaTokenizerFast(name_or_path='codellama/CodeLlama-13b-Instruct-hf', vocab_size=32016, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'additional_special_tokens': ['▁<PRE>', '▁<MID>', '▁<SUF>', '▁<EOT>']}, clean_up_tokenization_spaces=False)


In [11]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-13b-Instruct-hf", device_map="auto", load_in_8bit=True)
print(model)

bin c:\Users\TeamD\.conda\envs\sfm\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Loading checkpoint shards: 100%|██████████| 3/3 [01:27<00:00, 29.14s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear8bitLt(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear8bitLt(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear8bitLt(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear8bitLt(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear8bitLt(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
 

In [12]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

In [72]:
# run the test without any agent code to see what the fail error is
write_test(with_agent_content(read_test(), ""))
build_success, build_output = hot_reload()
if not build_success:
    print("Fail!")
    print(build_output)
else:
    print("Success!")
if build_success:
    test_output = run_test()
    print(f"Test output: {test_output}")

[Hot Reload] Focusing IntelliJ
[Hot Reload] Trigger hot reload
[Hot Reload] Wait for build to finish
[Hot Reload] Copy build output
[Hot Reload] Build output succeeded: True
Success!
[Run Test] Running test, ensure you ran `/sfm_ai listen` in Minecraft
Test output: open_door failed! Expected property open to be true, was false at 2,-58,4 (relative: 2,2,1) (t=60)


In [80]:
# Create the code that will be inserted after `// begin agent code` that will make the test pass. Your response MUST begin with ```java and end with ```. 
# You MUST only provide the code for the agent block, and MUST NOT retype the entire test file.
# You MUST propose a hypothesis in a comment at the beginning of the code that describes why the test is failing
prompt = f"""
[INST] The code for a failing test is provided. Make it pass. Your response MUST be 5 lines or fewer. [/INST]

Test output:
{test_output}

Test content:
```java
{with_agent_content(read_test(), '<FILL_ME> ONLY PROVIDE THIS PART, DO NOT RETYPE ALL THE CODE')}
```
""".strip() + "\n"
print(prompt)

[INST] The code for a failing test is provided. Make it pass. Your response MUST be 5 lines or fewer. [/INST]

Test output:
open_door failed! Expected property open to be true, was false at 2,-58,4 (relative: 2,2,1) (t=60)

Test content:
```java
package ca.teamdman.sfm.ai;

import ca.teamdman.sfm.SFM;
import ca.teamdman.sfm.SFMGameTestBase;
import net.minecraft.core.BlockPos;
import net.minecraft.gametest.framework.GameTest;
import net.minecraft.gametest.framework.GameTestHelper;
import net.minecraft.world.entity.item.ItemEntity;
import net.minecraft.world.item.ItemStack;
import net.minecraft.world.item.Items;
import net.minecraft.world.level.block.Blocks;
import net.minecraft.world.level.block.DoorBlock;
import net.minecraft.world.phys.Vec3;
import net.neoforged.gametest.GameTestHolder;
import net.neoforged.gametest.PrefixGameTestTemplate;

@GameTestHolder(SFM.MOD_ID)
@PrefixGameTestTemplate(false)
public class TestChambers extends SFMGameTestBase {
    /**
     * The agent must make 

In [81]:
response0 = pipe(prompt)[0]['generated_text']
print(response0)#.split("[/INST]")[1])

c:\Users\TeamD\.conda\envs\sfm\lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\TeamD\.conda\envs\sfm\lib\site-packages\transformers\generation\configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] The code for a failing test is provided. Make it pass. Your response MUST be 5 lines or fewer. [/INST]

Test output:
open_door failed! Expected property open to be true, was false at 2,-58,4 (relative: 2,2,1) (t=60)

Test content:
```java
package ca.teamdman.sfm.ai;

import ca.teamdman.sfm.SFM;
import ca.teamdman.sfm.SFMGameTestBase;
import net.minecraft.core.BlockPos;
import net.minecraft.gametest.framework.GameTest;
import net.minecraft.gametest.framework.GameTestHelper;
import net.minecraft.world.entity.item.ItemEntity;
import net.minecraft.world.item.ItemStack;
import net.minecraft.world.item.Items;
import net.minecraft.world.level.block.Blocks;
import net.minecraft.world.level.block.DoorBlock;
import net.minecraft.world.phys.Vec3;
import net.neoforged.gametest.GameTestHolder;
import net.neoforged.gametest.PrefixGameTestTemplate;

@GameTestHolder(SFM.MOD_ID)
@PrefixGameTestTemplate(false)
public class TestChambers extends SFMGameTestBase {
    /**
     * The agent must make 

In [64]:
agent_code0 = response0.split("```java")[1].split("```")[0].strip()
agent_code0 = agent_code0.replace("// begin agent code", " ")
agent_code0 = agent_code0.replace("// end agent code", " ").strip()
print(agent_code0)

var torchPos = new BlockPos(2, 3, 1);
helper.setBlock(torchPos, Blocks.REDSTONE_TORCH);


In [61]:
write_test(with_agent_content(read_test(), indent_text(agent_code0, 8)))

In [62]:
build_success, build_output = hot_reload()
if not build_success:
    print("Fail!")
    print(build_output)
else:
    print("Success!")

[Hot Reload] Focusing IntelliJ
[Hot Reload] Trigger hot reload
[Hot Reload] Wait for build to finish
[Hot Reload] Copy build output
[Hot Reload] Build output succeeded: True
Success!


In [63]:
if build_success:
    test_output = run_test()

[Run Test] Running test, ensure you ran `/sfm_ai listen` in Minecraft


In [ ]:
prompt = f"""
[INST] The assistant is tasked with solving puzzles in a Minecraft game test environment, similar to the video game _Portal_. The assistant is presented a game test with some code at the beginning and end of the test that the agent can not change. The agent is responsible for replacing the code in the middle of the test to cause the test to succeed. Write the code to be inserted after `// begin agent code` that will cause the test to pass. Create a hypothesis for why the test is not currently passing, then propose a solution wrapped using ```.
Current test content:
{with_agent_content(read_test(), '')}

Build output:
{"Success" if build_success else build_output}

Test output:
{"N/A" if not build_success else test_output}

[/INST]
""".strip() + "\n\n"
print(prompt)

In [106]:
fn0 = {
    "role": "function",
    "name": "run_test",
    "content": build_output if not build_success else test_output,
}
print(fn0)

{'role': 'function', 'name': 'run_test', 'content': 'D:\\Repos\\Minecraft\\Forge\\SuperFactoryManager\\src\\gametest\\java\\ca\\teamdman\\sfm\\ai\\TestChambers.java:96: error: method assertBlockState in class GameTestHelper cannot be applied to given types;\n                helper.assertBlockState(redstonePos, Blocks.REDSTONE_WIRE.defaultBlockState().setValue(PressurePlateBlock.POWERED, true));\n                      ^\n  required: BlockPos,Predicate<BlockState>,Supplier<String>\n  found:    BlockPos,BlockState\n  reason: actual and formal argument lists differ in length'}


In [107]:
response1 = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        *messages,
        response0.choices[0]["message"],
        fn0,
    ],
    functions=[
        {
          "name": "run_test",
          "description": "Run the test with the agent code block substituted for the provided content. Requesting a visual observation will slow down the process through a dependency on humans.",
          "parameters": RunTest.model_json_schema()
        },
    ],
    function_call={"name": "run_test"}
)
print(response1)

{
  "id": "chatcmpl-7x8FcWIAESmP2ehk4SgX63z6W2xsz",
  "object": "chat.completion",
  "created": 1694327424,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "run_test",
          "arguments": "{\n  \"include_visual_observation\": false,\n  \"agent_test_code\": \"// set the redstone dust\\nhelper.setBlock(redstonePos, Blocks.REDSTONE_WIRE);\\n\\n// add a tick delay before checking if the door is opened\\nhelper.runAfterDelay(1, () -> {\\n    helper.assertBlock(redstonePos, Blocks.REDSTONE_WIRE);\\n});\\n\\n// Spawn a player\\nLocalPlayer player = helper.spawnPlayer(helper.absolutePos(pressurePlatePos));\\n\\n// Drop the diamond on the pressure plate\\nplayer.drop(new ItemStack(Items.DIAMOND), true);\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1329,
    "completion_tokens": 128,
    "total_tokens": 145

In [108]:
if response1.choices[0]["message"]["content"] is not None:
    print(response1.choices[0]["message"]["content"])
if "function_call" in response1.choices[0]["message"]:
    run = RunTest(**json.loads(response1.choices[0]["message"]["function_call"]["arguments"]))
    write_test(with_agent_content(read_test(), indent_text(run.agent_test_code, 12)))
    build_success, build_output = hot_reload()
    if not build_success:
        print(build_output)

[Hot Reload] Focusing IntelliJ
[Hot Reload] Trigger hot reload
[Hot Reload] Wait for build to finish
[Hot Reload] Copy build output
[Hot Reload] Build output succeeded: False
D:\Repos\Minecraft\Forge\SuperFactoryManager\src\gametest\java\ca\teamdman\sfm\ai\TestChambers.java:96: error: no suitable method found for assertBlock(BlockPos,Block)
                helper.assertBlock(redstonePos, Blocks.REDSTONE_WIRE);
                      ^
    method GameTestHelper.assertBlock(BlockPos,Predicate<Block>,String) is not applicable
      (actual and formal argument lists differ in length)
    method GameTestHelper.assertBlock(BlockPos,Predicate<Block>,Supplier<String>) is not applicable
      (actual and formal argument lists differ in length)


In [109]:
if build_success:
    test_output = run_test()

In [110]:
fn1 = {
    "role": "function",
    "name": "run_test",
    "content": build_output if not build_success else test_output,
}

In [111]:
response2 = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        *messages,
        response0.choices[0]["message"],
        fn0,
        response1.choices[0]["message"],
        fn1
    ],
    functions=[
        {
          "name": "run_test",
          "description": "Run the test with the agent code block substituted for the provided content. Requesting a visual observation will slow down the process through a dependency on humans.",
          "parameters": RunTest.model_json_schema()
        },
    ],
)
print(response2)

{
  "id": "chatcmpl-7x8Fj1pwEaZ8dDxftHQo4cVhcydEo",
  "object": "chat.completion",
  "created": 1694327431,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "run_test",
          "arguments": "{\n  \"include_visual_observation\": false,\n  \"agent_test_code\": \"// set the redstone dust\\nhelper.setBlock(redstonePos, Blocks.REDSTONE_WIRE);\\n\\n// Spawn a player\\nLocalPlayer player = helper.spawnPlayer(helper.absolutePos(pressurePlatePos));\\n\\n// Drop the diamond on the pressure plate\\nplayer.drop(new ItemStack(Items.DIAMOND), true);\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 1594,
    "completion_tokens": 91,
    "total_tokens": 1685
  }
}


In [112]:
save_convo([
    *messages,
    response0.choices[0]["message"],
    fn0,
    response1.choices[0]["message"],
    fn1,
    response2.choices[0]["message"],
])